In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import output_file
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.InteractiveDrawing.bokeh.bokehInteractiveParameters import *
from RootInteractive.Tools.pandaTools import initMetadata
import pandas as pd
import numpy as np
import math
import logging

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
path = "../output/"
file = "run456060805_221112060811_clusters.csv"
df = pd.read_csv(path+file)
initMetadata(df)
df.meta.metaData = {    
    "localX.AxisTitle":      "local X (mm)",
    "localY.AxisTitle":      "local Y (mm)",
    "globalX.AxisTitle":     "global X (mm)",
    "globalY.AxisTitle":     "global Y (mm)",
    "globalZ.AxisTitle":     "global Z (mm)",
    "ID.AxisTitle":          "ALPIDE",
    "nPixels.AxisTitle":     "Number of pixels in cluster",
    "columnWidth.AxisTitle": "Cluster width in columns (Width in X)",
    "rowWidth.AxisTitle": "Cluster width in rows (Width in Y)"
}

In [ ]:
myMap = {"ALPIDE_0":0,"ALPIDE_1":1,"ALPIDE_2":2,"ALPIDE_3":3,"ALPIDE_4":4}
df["ID"] = df["detector"].map(myMap)
variables=list([i for i in df.columns if "detector" not in i])
df.head()

In [ ]:
aliasArray = [
    #("ALPIDE_bitmap","0x1*(detector==\"ALPIDE_0\")")
]

aliasArray+=[
    {
        "name": "customFunction",
        "variables": [i for i in variables if  "ustom" not in i ],
        "func":"customFunction",
    }
]

In [ ]:

axisTransforms = [None, "sqrt", "lambda x: log(1+x)","lambda x: 1/sqrt(x)"]

parameterArray = [
    # Parameters to enable custom histogramming
    {"name": "varX", "value":"localX", "options":variables},
    {"name": "varY", "value":"localY", "options":variables},
    {"name": "varZ", "value":"nPixels", "options":variables},
    {"name": "nbinsX", "value":30, "range":[10, 200]},
    {"name": "nbinsY", "value":30, "range":[10, 200]},
    {"name": "nbinsZ", "value":5, "range":[1,10]},
    {"name": "varColor", "value":"nPixels", "options":variables},
    # Transformations on axes
    {'name': "xAxisTransform", "value":None, "options":axisTransforms},
    {'name': "yAxisTransform", "value":None, "options":axisTransforms},
    {'name': "zAxisTransform", "value":None, "options":axisTransforms},
    # custom selection
    {'name': 'customFunction', "value":"return 1"},
]

parameterArray.extend(figureParameters["legend"]['parameterArray'])   
parameterArray.extend(figureParameters["markers"]['parameterArray'])  

In [ ]:
widgetParams = [
    # Selection on data
    ["range", ["localX"]],
    ["range", ["localY"]],
    ["range", ["globalX"]],
    ["range", ["globalY"]],
    ["range", ["globalZ"]],
    ["multiSelect", ["detector"]],
    ["range", ["eventNr"]],
    ["range", ["timeStamp"]],
    ["range", ["charge"]],
    ["range", ["nPixels"]],
    ["range", ["columnWidth"]],
    ["range", ["rowWidth"]],
    ["multiSelect", ["split"]],
    ['text', ['customFunction'], {"name": "Custom selection"}],
    # Selection for visualization
    ['select', ['varX'], {"name": "varX"}],
    ['select', ['varY'], {"name": "varY"}],
    ['select', ['varZ'], {"name": "varZ"}],
    ['slider', ['nbinsX'], {"name": "nbinsX"}],
    ['slider', ['nbinsY'], {"name": "nbinsY"}],
    ['slider', ['nbinsZ'], {"name": "nbinsZ"}],
    ['select', ['xAxisTransform'], {"name": "xAxisTransform"}],
    ['select', ['yAxisTransform'], {"name": "yAxisTransform"}],
    ['select', ['zAxisTransform'], {"name": "zAxisTransform"}],
    ['select', ['varColor'], {"name": "varColor"}],
]

widgetParams.extend(figureParameters["legend"]["widgets"])
widgetParams.extend(figureParameters["markers"]["widgets"])

widgetLayoutDesc = {
    "Selection": [[0,1,2,3,4],[5,6,7,8],[9,10,11,12],[13],{"sizing_mode": "scale_width"}],
    "Plotting" : [[14,15,16],[17,18,19],[20,21,22],[23],{"sizing_mode": "scale_width"}],
    "Legend": figureParameters['legend']['widgetLayout'],
    "Markers":["markerSize"]
} 

figureGlobalOption={}
figureGlobalOption=figureParameters["legend"]["figureOptions"]
figureGlobalOption["size"]="markerSize"
figureGlobalOption["x_transform"]="xAxisTransform"
figureGlobalOption["y_transform"]="yAxisTransform"
figureGlobalOption["z_transform"]="zAxisTransform"

In [ ]:
histoArray = [
        {"name": "histo2Dhitmap", "variables": ["localX","localY"], "nbins":["nbinsX","nbinsY"], "axis":[1],"unbinned_projections":True},
        {"name": "histo3Dcustom", "variables": ["varX","varY","varZ"], "nbins":["nbinsX","nbinsY","nbinsZ"],"axis":[1,2],"unbinned_projections":True},
        {"name": "histo2Dcustom", "variables": ["varX","varY"], "nbins":["nbinsX","nbinsY"], "axis":[1],"unbinned_projections":True},
        {"name": "histo1Dcustom", "variables": ["varX"], "nbins":"nbinsX"},

    ]

figureArray = [
    # 3D plot
    [["globalZ"], ["globalX"], {"color": "red", "size": 2, "varZ": "globalY", "colorZvar": "varColor", "name":"3D"}],
    # Hitmap
    [[("bin_bottom_0", "bin_top_0")], [("bin_bottom_1", "bin_top_1")], {"colorZvar":"bin_count","source":"histo2Dhitmap","name":"2Dhitmap"}],
    # Custom histogramming
    [["bin_center"],["bin_count"],{"source":"histo1Dcustom"}],
    [[("bin_bottom_0", "bin_top_0")], [("bin_bottom_1", "bin_top_1")], {"colorZvar":"bin_count","source":"histo2Dcustom"}],
    [[("bin_bottom_0", "bin_top_0")], [("bin_bottom_1", "bin_top_1")], {"colorZvar": "mean", "source":"histo3Dcustom_2"}],
    #
    figureGlobalOption
]

In [ ]:
layout = {
    "Geometry" : [[0],{"plot_height":500,"sizing_mode":"fixed"}],
    "Hitmap" : [[1],{"plot_height":500,"sizing_mode":"fixed"}],
    "Custom" : [[2,3,4],{"plot_height":500}]
}

In [ ]:
tooltips = []
output_file("../output/"+file[0:-4]+".html")
arrayCompressionRelative10=[(".*",[("relative",10), ("code",0), ("zip",0), ("base64",0)])]
arrayCompression=arrayCompressionRelative10
bokehDrawSA.fromArray(df, "eventNr>=0", figureArray, widgetParams, layout=layout, tooltips=tooltips, widgetLayout=widgetLayoutDesc,
                      histogramArray=histoArray,aliasArray=aliasArray,arrayCompression=arrayCompression,parameterArray=parameterArray,
                      sizing_mode='scale_width',rescaleColorMapper=True,)